In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate
import os
from sklearn.model_selection import train_test_split
import pickle
from surprise import accuracy
from surprise import accuracy, Dataset, Reader, SVD, KNNBaseline
from surprise.model_selection import PredefinedKFold
import pprint
import tempfile
from typing import Dict, Text
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import defaultdict
from scipy import stats

In [2]:
#!pwd

/Users/alecclarkfeather/capstone_project


In [3]:
PATH = '/Users/alecclarkfeather/capstone_project/'

In [4]:
#sample = pd.read_csv(PATH + 'hit_rate_folds_actual/train_hr0.csv', header=None)
#sample.head()

,0,1,2
0,826574,9729,4.0
1,2200645,1719,4.0
2,2150434,5356,5.0
3,2142065,8753,3.0
4,867086,6721,4.0


In [5]:
#sample.shape

(2333222, 3)

In [6]:
hr_results_df = pd.read_csv(PATH + 'hr_results_df.csv')
rmse_results_df = pd.read_csv(PATH + 'rmse_results_df.csv')

In [7]:
hr_results_df

,baseline,cf,svd
0,0.296561,0.439228,0.458413
1,0.297034,0.439001,0.460726
2,0.297715,0.438900,0.459328
3,0.298997,0.437977,0.460333
4,0.295871,0.439655,0.460662
5,0.298046,0.438874,0.459719
6,0.297368,0.439560,0.461304
7,0.298030,0.439275,0.460432
8,0.296864,0.438533,0.459555
9,0.298423,0.439623,0.459121


In [8]:
rmse_results_df

,baseline,cf,svd
0,1.478618,1.066947,1.006683
1,1.477799,1.067403,1.006872
2,1.478326,1.067594,1.007039
3,1.478533,1.067073,1.007092
4,1.478335,1.066790,1.007514
5,1.477217,1.066989,1.006884
6,1.477425,1.067445,1.006484
7,1.478469,1.067561,1.007107
8,1.477504,1.067057,1.007069
9,1.478085,1.067257,1.006919


In [13]:
rmse_results_df['baseline'].iloc[:2].values[1]

1.477799085109755

In [30]:
def five_by_two_paired_t_test(df, col1, col2):
    variances = []
    diffs = []
    for i in range(2, 12, 2):
        group1 = df[col1].iloc[:i].values
        group2 = df[col2].iloc[:i].values
        result1a = group1[i-2]
        result1b = group1[i-1]
        result2a = group2[i-2]
        result2b = group2[i-1]
        performance1 = result1a - result2a
        performance2 = result1b - result2b
        mean_performance = (performance1 + performance2) / 2
        variance_performance = (performance1 - mean_performance)**2 + (performance2 - mean_performance)**2
        variances.append(variance_performance)
        diffs.append(performance1)
        
    t_statistic = diffs[0] / np.sqrt((1/len(variances))*np.sum(variances))
    p_value = stats.t.sf(abs(t_statistic), df=len(variances))
    
    return t_statistic, p_value
        

In [31]:
t_statistic, p_value = five_by_two_paired_t_test(rmse_results_df, 'baseline', 'cf')

In [20]:
#rmse_results_df

,baseline,cf,svd
0,1.478618,1.066947,1.006683
1,1.477799,1.067403,1.006872
2,1.478326,1.067594,1.007039
3,1.478533,1.067073,1.007092
4,1.478335,1.066790,1.007514
5,1.477217,1.066989,1.006884
6,1.477425,1.067445,1.006484
7,1.478469,1.067561,1.007107
8,1.477504,1.067057,1.007069
9,1.478085,1.067257,1.006919


In [32]:
t_statistic, p_value

(588.3746395042142, 1.345831530364664e-13)

In [26]:
#diff_test

[0.411670735709343,
 0.41073179188007924,
 0.41154476245618166,
 0.4099797285168356,
 0.4104470085509342]

In [27]:
#t_statistic

588.3746395042142

In [29]:
#stats.t.sf(abs(t_statistic), df=5)

1.345831530364664e-13

In [33]:
t_stats_rmse = []
p_values_rmse = []

t_b_vs_cf, p_b_vs_cf = five_by_two_paired_t_test(rmse_results_df, 'baseline', 'cf')
t_b_vs_svd, p_b_vs_svd = five_by_two_paired_t_test(rmse_results_df, 'baseline', 'svd')
t_cf_vs_svd, p_cf_vs_svd = five_by_two_paired_t_test(rmse_results_df, 'cf', 'svd')

t_stats_rmse.append(t_b_vs_cf)
t_stats_rmse.append(t_b_vs_svd)
t_stats_rmse.append(t_cf_vs_svd)

p_values_rmse.append(p_b_vs_cf)
p_values_rmse.append(p_b_vs_svd)
p_values_rmse.append(p_cf_vs_svd)

In [34]:
t_stats_rmse

[588.3746395042142, 1058.5433431916879, 157.2509913257679]

In [35]:
p_values_rmse

[1.345831530364664e-13, 7.140464865047784e-15, 9.865492033175221e-11]

In [36]:
t_test_results_dict = dict(t_statistic=t_stats_rmse, p_value_unadjusted=p_values_rmse)
t_test_results_dict

{'t_statistic': [588.3746395042142, 1058.5433431916879, 157.2509913257679],
 'p_value_unadjusted': [1.345831530364664e-13,
  7.140464865047784e-15,
  9.865492033175221e-11]}

In [37]:
t_test_results_rmse_df = pd.DataFrame(t_test_results_dict, index=['baseline_vs_cf',
                                                                 'baseline_vs_svd',
                                                                 'cf_vs_svd'])
t_test_results_rmse_df

,t_statistic,p_value_unadjusted
baseline_vs_cf,588.374640,1.345832e-13
baseline_vs_svd,1058.543343,7.140465e-15
cf_vs_svd,157.250991,9.865492e-11


In [38]:
t_stats_hr = []
p_values_hr = []

t_b_vs_cf, p_b_vs_cf = five_by_two_paired_t_test(hr_results_df, 'baseline', 'cf')
t_b_vs_svd, p_b_vs_svd = five_by_two_paired_t_test(hr_results_df, 'baseline', 'svd')
t_cf_vs_svd, p_cf_vs_svd = five_by_two_paired_t_test(hr_results_df, 'cf', 'svd')

t_stats_hr.append(t_b_vs_cf)
t_stats_hr.append(t_b_vs_svd)
t_stats_hr.append(t_cf_vs_svd)

p_values_hr.append(p_b_vs_cf)
p_values_hr.append(p_b_vs_svd)
p_values_hr.append(p_cf_vs_svd)

In [39]:
t_stats_hr

[-115.72996706768873, -115.89031618960854, -16.915945970664175]

In [40]:
p_values_hr

[4.5676977700525175e-10, 4.5361951459958135e-10, 6.601872454475318e-06]

In [41]:
t_test_results_dict_hr = dict(t_statistic=t_stats_hr, p_value_unadjusted=p_values_hr)
t_test_results_dict_hr

{'t_statistic': [-115.72996706768873,
  -115.89031618960854,
  -16.915945970664175],
 'p_value_unadjusted': [4.5676977700525175e-10,
  4.5361951459958135e-10,
  6.601872454475318e-06]}

In [42]:
t_test_results_hr_df = pd.DataFrame(t_test_results_dict_hr, index=['baseline_vs_cf',
                                                                 'baseline_vs_svd',
                                                                 'cf_vs_svd'])
t_test_results_hr_df

,t_statistic,p_value_unadjusted
baseline_vs_cf,-115.729967,4.567698e-10
baseline_vs_svd,-115.890316,4.536195e-10
cf_vs_svd,-16.915946,6.601872e-06


In [43]:
t_test_results_rmse_df.to_csv(PATH + 't_test_results_rmse_df.csv', index=False)
t_test_results_hr_df.to_csv(PATH + 't_test_results_hr_df.csv', index=False)

In [ ]:
def get_top_n_tf(predictions, n=100):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [ ]:
PATH = 'rmse/hit_rate_folds_actual/'

hit_rates_nn = []

for i in range(5):
    train_file1 = PATH + f'train_hr{i}.csv'
    test_file1 = PATH + f'test_hr{i}.csv'
    df_train1 = pd.read_csv(train_file1, header=None)
    df_test1 = pd.read_csv(test_file1, header=None)
    df_train1.columns = ['user_id', 'movie_id', 'rating']
    df_test1.columns = ['user_id', 'movie_id', 'rating']
    
    deep_model1 = NetflixRmseModel()
    deep_model1.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
    
    deep_model2 = NetflixRmseModel()
    deep_model2.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
    
    df_train_tf1 = tf.data.Dataset.from_tensor_slices(dict(df_train1))
    df_test_tf1 = tf.data.Dataset.from_tensor_slices(dict(df_test1))
    
    df_train_tf1 = df_train_tf1.map(lambda x: {'user_id': x['user_id'],
    'movie_id': x['movie_id'],
    'rating': x['rating']})
    
    df_test_tf1 = df_test_tf1.map(lambda x: {'user_id': x['user_id'],
    'movie_id': x['movie_id'],
    'rating': x['rating']})
    
    df_train_tf1 = df_train_tf1.batch(8192).cache()
    df_test_tf1 = df_test_tf1.batch(8192).cache()
    
    history_baseline1 = deep_model1.fit(df_train_tf1, epochs=10)
    history_baseline2 = deep_model2.fit(df_test_tf1, epochs=10)
    
    num_users1 = 0
    num_users2 = 0
    num_hits1 = 0
    num_hits2 = 0
    
    predictions_1a = defaultdict(list)

    for i, row in df_test1.iterrows():
        user = row['user_id']
        movie = row['movie_id']
        predicted_rating = deep_model1({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
        predictions_1a[user].append((movie, predicted_rating[0][0]))
        
    for user_id, user_ratings in predictions_1a.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        predictions_1a[user_id] = user_ratings[:100]
        
    predictions_1b = defaultdict(list)

    for i, row in df_train1.iterrows():
        user = row['user_id']
        movie = row['movie_id']
        predicted_rating = deep_model2({'user_id': np.array([user]),
                                           'movie_id': np.array([movie])})
        predictions_1b[user].append((movie, predicted_rating[0][0]))
        
    for user_id in predictions_1a.keys():
        top_movie = top_ratings_df[top_ratings_df['user_id']==user_id]['movie_id']
        num_users1 += 1
        num_recs = len(predictions_1a[user_id])
        top_number = int(num_recs*0.25)
        boundary_for_user = [rec[1] for rec in predictions_1a[user_id]][top_number]
        pred_top_movie = deep__model1({'user_id': np.array([user_id]),
                                         'movie_id': np.array([top_movie])})
        if pred_top_movie[0][0] > boundary_for_user:
            num_hits1 += 1
            
    for user_id in predictions_1b.keys():
        top_movie = top_ratings_df[top_ratings_df['user_id']==user_id]['movie_id']
        num_users2 += 1
        num_recs = len(predictions_1b[user_id])
        top_number = int(num_recs*0.25)
        boundary_for_user = [rec[1] for rec in predictions_1b[user_id]][top_number]
        pred_top_movie = deep_model2({'user_id': np.array([user_id]),
                                         'movie_id': np.array([top_movie])})
        if pred_top_movie[0][0] > boundary_for_user:
            num_hits2 += 1
            
            
    hit_rates_nn.append(num_hits1 / num_users1)
    hit_rates_nn.append(num_hits2 / num_users2)
    
    